# Embeddings Quantization

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import uuid
import pandas as pd
from datasets import load_dataset
from fastembed import TextEmbedding
from qdrant_client import models, QdrantClient

from src import load_data, embeddings


c:\Users\manua\Documents\repos\dslabs\dslab-rag-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Params

In [4]:
RETRIEVE_K=5
SMPL_SIZE = 25  # int or None

# Clients

In [5]:
client_qdrant = QdrantClient(":memory:") 


# Load and Prepare Data

In [6]:
df_ragquas = load_data.ragquas()
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [7]:
if SMPL_SIZE:
    df_docs_smpl = df_docs.iloc[:SMPL_SIZE]
else:
    df_docs_smpl = df_docs

# Embeddings

In [8]:
encoder = TextEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    )


dn_embeddings = list(encoder.embed(df_docs_smpl['text_']))

print(f"{len(dn_embeddings)=}")
print(f"{len(dn_embeddings[0])=}")

C:\Users\manua\AppData\Local\Temp\ipykernel_19224\315642572.py:1: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  encoder = TextEmbedding(


len(dn_embeddings)=25
len(dn_embeddings[0])=768


In [9]:
lst_embeddings = []

for idx in range(len(dn_embeddings)):
    dn_vec = dn_embeddings[idx]

    lst_embeddings.append(
        dn_vec.tolist()
    )

lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs_smpl,
)

lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)

print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=25


In [10]:
client_qdrant.create_collection(
    collection_name='raw',
    vectors_config=models.VectorParams(
            size=encoder.embedding_size, 
            distance=models.Distance.COSINE,
        )
)

client_qdrant.create_collection(
    collection_name='q2',
    vectors_config=models.VectorParams(
            size=encoder.embedding_size, 
            distance=models.Distance.COSINE,
        ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(
            always_ram=True,
        ),
    ),
)

client_qdrant.upsert(
    collection_name='raw',
    points=lst_qdrant_pts
)

client_qdrant.upsert(
    collection_name='q2',
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [11]:


client_qdrant.upsert(
    collection_name='q2',
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [12]:
_q_text = df_docs_smpl.sample(1)['question']

In [13]:
_q_vec = next(encoder.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        "raw",
        _q_vec,
        retrieve_k=2,
)

Question: 18    Hola! ¿Podrías explicarme cuáles son los tres ...
Name: question, dtype: object
point.id='591405f4-b0f1-4154-808a-c94885f6633e'
point.score=0.5694318257579751
Doc: Surya Namaskar: la mejor forma de empezar el día

El Saludo al Sol o Surya Namaskar es una de las prácticas más conocidas en yoga.

Todavía recuerdo cuando lo practiqué por primera vez… ¡vaya lío!

Uno tiene que poner a prueba su concentración y estar completamente presente para poder seguir el ritmo (¡y no perderse por el camino!).

Eso sí, se aprende rápido y, una vez aprendido, difícilmente se olvida.

Además, debo confesar que fue, y sigue siendo, una de mis prácticas de yoga favoritas.
¿Qué...
idx: 16, j: 3
------------------------------
point.id='a2ca462e-35a3-4b59-8166-1b170b3198f4'
point.score=0.5549750366460964
Doc: Saludo al sol o Surya Namaskar
Por
ManuelMM
Saludo-al-sol

El Saludo al sol, también llamado Surya Namaskar, es una secuencia de 12 posturas que se repiten varias veces. Cada movimiento e

In [14]:
_q_vec = next(encoder.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        "q2",
        _q_vec,
        retrieve_k=2,
)

Question: 18    Hola! ¿Podrías explicarme cuáles son los tres ...
Name: question, dtype: object
point.id='591405f4-b0f1-4154-808a-c94885f6633e'
point.score=0.5694318227618158
Doc: Surya Namaskar: la mejor forma de empezar el día

El Saludo al Sol o Surya Namaskar es una de las prácticas más conocidas en yoga.

Todavía recuerdo cuando lo practiqué por primera vez… ¡vaya lío!

Uno tiene que poner a prueba su concentración y estar completamente presente para poder seguir el ritmo (¡y no perderse por el camino!).

Eso sí, se aprende rápido y, una vez aprendido, difícilmente se olvida.

Además, debo confesar que fue, y sigue siendo, una de mis prácticas de yoga favoritas.
¿Qué...
idx: 16, j: 3
------------------------------
point.id='a2ca462e-35a3-4b59-8166-1b170b3198f4'
point.score=0.5549750365408745
Doc: Saludo al sol o Surya Namaskar
Por
ManuelMM
Saludo-al-sol

El Saludo al sol, también llamado Surya Namaskar, es una secuencia de 12 posturas que se repiten varias veces. Cada movimiento e